# Si.T.T. Basic Example

In this notebook, we will explore the basic features of Si.T.T. and show how to make the platform work on a basic level.

## Installing Si.T.T.

First, we need to integrate the Python library. Note, you need to install the required Python packages, first. A common way to do this is to use pip (search the Web for venv, Conda etc. for specific environments).

This is pretty straightforward (upgrade will ensure you have the latest version of Si.T.T. installed):

In [ ]:
# uncomment to install
# !pip install --upgrade sitt

If you want to uninstall Si.T.T., you can do so by running the following command:

```console
pip uninstall -y sitt
```

## Basic Usage

After you have installed Si.T.T., you can import the library (we will also import a couple if other modules):

In [1]:
import sitt

## Configuration

In order to run a simulation, you need a configuration. The configuration defines all the variables, modules and simulation steps needed. For the sake of demonstration, we will create the configuration using Python.

In [ ]:
config = sitt.Configuration()
print(config)

As you can see, the configuration is a simple Python object. There are three main components of the simulation:

* Preparation: prepare data
* Simulation: actual simulation
* Output: showing or saving data

You can read more on these components in the [Concept Readme](../readmes/concept.md).


## Preparation

For now, let us look at the first part, the preparation. We will skip the simulation and output steps, and import a serialized graph (containing an arbitrary set of hubs and roads in Southern Austria):

In [ ]:
# Skip simulation (and output)
config.skip_step = sitt.SkipStep.SIMULATION

# we will also make the output verbose, so we can see what the application is doing in its log output.
config.verbose = True

# Create an import module for the preparation component and add it the preparation component
graph_load = sitt.preparation.GraphLoad(filename='example_graph.pkl')
config.preparation.append(graph_load)

To see what is in the graph, we can use another module:

In [ ]:
preparation_debug = sitt.preparation.DebugDisplayRoadsAndHubs()
config.preparation.append(preparation_debug)

Now let us run the application (the preparation, that is) and see what the debug output will show us. The simulation can be run by creating a core object and running it:

In [ ]:
core = sitt.Core(config)
core.run()

You can see the network... kind of, the image is a bit too small, but you get a first impression. Now let us check the data a bit more thoroughly. What paths could we take from point A to point B and how would the roads look like?

No problem, let us update the debug and just output all possible paths between Lienz and Villach:

In [ ]:
# turn off showing whole network
preparation_debug.draw_network = False

# set start and end points
preparation_debug.start = 'LIENZ'
preparation_debug.end = 'VILLACH'

core.run()

Wow, there six possible routes including height profiles. Time to start the simulation!

## A Simple Simulation

Now we want to actually run the simulation. Like above, we have to create a configuration first. In addition to the things shown above, we will add a preparation component to precalculate the possible routes used in the simulation.

In [ ]:
# create new config
config = sitt.Configuration()

# Skip output
config.skip_step = sitt.SkipStep.OUTPUT

# we will also make the output verbose, so we can see what the application is doing in its log output.
config.verbose = True

# We want to tell our simulation where to start and end the simulation:
config.simulation_start = 'LIENZ'
config.simulation_end = 'VILLACH'

# Create an import module for the preparation component and add it the preparation component
graph_load = sitt.preparation.GraphLoad(filename='example_graph.pkl')
config.preparation.append(graph_load)

# Before we can run our simulation, we need to create all possible routes - here is a simple route creator
create_routes = sitt.preparation.CreateRoutes()
config.preparation.append(create_routes)

We will use a very simple simulation runner: An agent travelling 4 kph up to a maximum of 8 hours a day (so 32 km maximum which is a good average for a normal person on foot). The agent will slow a bit on slopes, we will use a linear factor to simulate this. Granted, this is not a very good simulation (some exponential function would be more precise), but for the sake of simplicity, it will do.

In [ ]:
# This is the new part: Create a simple simulation stepper
simple_stepper = sitt.simulation_step.Simple(speed=4.0)
config.simulation_step.append(simple_stepper)

# Set start and end points of the simulation
config.simulation_start = 'LIENZ'
config.simulation_end = 'VILLACH'

# Run the core
core = sitt.Core(config)
core.run()

As you can experiment a bit, changing values or reversing the order of the simulation start and end. If you change the values too much, you might end up with paths that are cancelled due to the agent not being able to traverse this particular path.

It is also possible to chain steppers. This does not make sense with the simple stepper, but there are use cases where environmental changes might change travel times, and you want to make this modular.

## Simulation Output

The last step showed you how to run a basic simulation. We are still missing some output we can analyze. This can be done by adding output modules. Like preparation and simulation modules, you can chain output modules to create multiple outputs at once.

We will expand our simple simulation and add a JSON output module:

In [2]:
# create new config
config = sitt.Configuration()

# We will make out application quiet this time, we do not need any debug output.
config.quiet = True

# We want to tell our simulation where to start and end the simulation:
config.simulation_start = 'LIENZ'
config.simulation_end = 'VILLACH'

# Create an import module for the preparation component and add it the preparation component
graph_load = sitt.preparation.GraphLoad(filename='example_graph.pkl')
config.preparation.append(graph_load)

# Before we can run our simulation, we need to create all possible routes - here is a simple route creator
create_routes = sitt.preparation.CreateRoutes()
config.preparation.append(create_routes)

# Create a simple simulation stepper
simple_stepper = sitt.simulation_step.Simple(speed=4.0)
config.simulation_step.append(simple_stepper)

# New: Add the JSON output module - do not convert output data to string, leave it as it is
json_output = sitt.output.JSONOutput(to_string=False)
config.output.append(json_output)

# Set start and end points of the simulation
config.simulation_start = 'LIENZ'
config.simulation_end = 'VILLACH'

# Run the core
core = sitt.Core(config)
outputs = core.run()

print(len(outputs))

1


As we can see, `core.run()` has a return value: A list of outputs, equal to the number of output modules defined in the configuration. So our output will be in `outputs[0]`. It is quite longish, so we will check the keys first:

In [3]:
list(outputs[0].keys())

['simulation_start',
 'simulation_end',
 'agents_finished',
 'agents_cancelled',
 'legs',
 'nodes',
 'paths']

Show the agents:

In [4]:
outputs[0]['agents_finished']

[{'uid': '7s5Azwi5VRlS',
  'uids': ['7s5Azwi5VRlS',
   'C3vYdaHCIDhz',
   'aia5kIohPSpa',
   'fcYv2rrUgWWr',
   'E5iecvSaNg01'],
  'status': 'finished',
  'day': 5,
  'hour': 11.499848697791732},
 {'uid': 'TFt372vtWW2R',
  'uids': ['TFt372vtWW2R',
   'C3vYdaHCIDhz',
   'aia5kIohPSpa',
   'fcYv2rrUgWWr',
   'E5iecvSaNg01'],
  'status': 'finished',
  'day': 5,
  'hour': 13.09813788939288}]

In [59]:
import json

def get_output():
    return json.dumps(outputs[0])

In [60]:
%%javascript
// The output of get_data will be stored in the following function
window.outputVar = null

IPython.notebook.kernel.execute(
    "get_output()", 
    {
        iopub: {
            output: function(response) {
                // Get output as plain text
                var output = response.content.data["text/plain"];
                // Remove unwanted characters
                output = output.substring(1, output.length-1).replace("\\'","'")
                // Set the variable
                 window.outputVar = JSON.parse(output)
            }
        }
    },
    {
        silent: false, 
        store_history: false, 
        stop_on_error: true
    }
)

<IPython.core.display.Javascript object>

In [61]:
%%javascript
console.log(window.outputVar)

<IPython.core.display.Javascript object>

In [ ]:
# uncomment to install leaflet and widgets for Jupyter
# !pip install ipyleaflet ipywidgets

In [7]:
from ipyleaflet import (Map, LayerGroup, basemaps, basemap_to_tiles, Marker, Polyline, Popup)
import ipywidgets as widgets
from IPython.display import display

In [8]:
# Create markers
markers = LayerGroup()
points = []

for node in outputs[0]['nodes']:
    message = widgets.HTML()
    message.value = f"<b>{node['id']}</b><br />Overnight: {node['overnight']}"
    
    # note that leaflet needs coordinates in lat, lng order
    marker = Marker(location=[node['geom']['coordinates'][1], node['geom']['coordinates'][0]], draggable=False,
                   title=node['id'])
    markers.add_layer(marker)
    # , popup=message
    
    points.extend([node['geom']['coordinates'][1], node['geom']['coordinates'][0]])
    
# create bounding box from array values - stolen from gbbox
min_x = min(points[::2])
min_y = min(points[1::2])
max_x = max(points[::2])
max_y = max(points[1::2])
padding = 0.025

bbox = ((min_x - padding, min_y - padding), (max_x + padding, max_y + padding))
center = [(min_x + max_x) / 2, (min_y + max_y) / 2]

In [9]:
# Create lines
paths = LayerGroup()

html = widgets.HTML()
def get_mouseover(path, line):
    def callback(*args, **kwargs):
        line.color="green"
    
    return callback
        
def get_mouseout(path, line):
    def callback(*args, **kwargs):
        line.color="#0033FF"
    
    return callback


for path in outputs[0]['paths']:
    message = widgets.HTML()
    message.value = f"<b>{path['id']}</b>"
    
    # note that leaflet needs coordinates in lat, lng order
    lats = [coord[1] for coord in path['geom']['coordinates']]
    lngs = [coord[0] for coord in path['geom']['coordinates']]
    coords = []
    
    for i in range(len(lats)):
        coords.append([lats[i], lngs[i]])
        
    line = Polyline(locations=coords, popup=message)
    line.on_mouseover(get_mouseover(path, line))
    line.on_mouseout(get_mouseout(path, line))
    paths.add_layer(line)

In [10]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=center,
    zoom=9
    )
m.fit_bounds(bbox)

m.add_layer(paths)
m.add_layer(markers)

m

Map(center=[46.77646547616253, 13.309529304531788], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [14]:
output = widgets.Output()

def get_button_clicked_callback(agent):
    def callback(*args, **kwargs):
        print(args)
        with output:
            print(agent)

    return callback

for agent in outputs[0]['agents_finished']:
    print(f"Agent {agent['uid']}, finished day {agent['day']}, hour day {agent['hour']}")
    
    button = widgets.Button(description='Show iteration')
    display(button, output)
                  
    button.on_click(get_button_clicked_callback(agent))

Agent 7s5Azwi5VRlS, finished day 5, hour day 11.499848697791732


Button(description='Show iteration', style=ButtonStyle())

Output()

Agent TFt372vtWW2R, finished day 5, hour day 13.09813788939288


Button(description='Show iteration', style=ButtonStyle())

Output()

(Button(description='Show iteration', style=ButtonStyle()),)


In [15]:
play = widgets.Play(
    value=50,
    min=0,
    max=100,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])